# Differentiable Lotto Implementation
(based on the "Open-ended Learning in Symmetric Zero-sum Games" paper)

## 1. Introduction


**Differentiable Lotto** is inspired by continuous Lotto (Hart, S. Discrete Colonel Blotto and General Lotto games. Int J Game Theory, 36:441–460, 2008.). The game is defined over a fixed set $C$ of $c$ 'customers', each being a point in $\mathbb{R}^2$. An agent’s strategy $(p, v) = \{(p_1,v_1), ... , (p_k,v_k)\}$, distributes one unit of mass over $k$ servers, where each server is a point $v_i \in \mathbb{R}^2$. Roughly, given the strategies of two players, $(p,v)$, $(q,w)$, customers are softly assigned to the nearest servers, determining the agents’ payoffs.

More formally, the payoff is

$$
\phi((p,v), (q,w)) := \sum_{i,j=1}^{c,k} (p_j v_{ij} - q_j w_{ij}),$$

where the scalars $v_{ij}$ and $w_{ij}$ depend on the distance between customer $i$ and the servers:

$$(v_{i1}, ... , w_{ik}) := softmax(-||c_i - v_1||^2, ... , -||c_i - w_k||^2),$$

where $c_i$ is the coordinate of customer $i$.

The width of a cloud of points is the expected distance from the barycenter. We impose agents to have width equal one. We use gradient ascent as our oracle.

For this implmentation,

- $c = 9$ customers chosen uniformly at random in the square $[-1,1]^2$
- $k = 2$ servers
- $n = 500$ games

## 2. Code Implementation of the game

### 2.0 Importing all necessary packages

In [1]:
import numpy as np
from random import randint
from scipy.optimize import linprog
import pickle
import matplotlib
import matplotlib.pyplot as plt

----------------

### 2.1 Initialization of initial strategies and fixed c customers

*random_point(lb = -1, ub = 1, n = 2)*:
- Input: Lower bound (lb), upper bound (ub) of each coordinate in $\mathbb{R}^n$ space
- Output: random point within the space

In [2]:
def random_point(lb = -1, ub = 1, n = 2):
    point = np.zeros((n,))
    for i in range(n):
        point[i] = np.random.uniform(lb, ub)
    return point

In [3]:
random_point()

array([-0.53919907, -0.17009413])

*random_points_list(lb = -1, ub = 1, n = 2, k = 2)*:
- Input: $k$ number of points with lower bound (lb), upper bound (ub) of each coordinate in $\mathbb{R}^n$ space
- Output: list of $k$ random points within the space

In [4]:
def random_points_list(lb = -1, ub = 1, n = 2, k = 9):
    points_list = []
    for i in range(k):
        points_list.append(random_point(lb,ub,n))
    points_list = np.asarray(points_list)
    return points_list

In [5]:
random_points_list(k=9)

array([[ 0.91006997,  0.39798922],
       [ 0.44573214,  0.97428297],
       [-0.91783279, -0.40232876],
       [-0.21621169,  0.17056842],
       [ 0.94935523, -0.96998382],
       [ 0.83829251,  0.86189049],
       [-0.60552187, -0.09345452],
       [-0.18799115, -0.82412405],
       [-0.25133223,  0.32359856]])

*prob_dist(k = 2)*:
- Input: $k$
- Output: $k$ numbers that sum up to 1

In [6]:
def prob_dist(k = 3):
    pd = np.random.uniform(0,1,k)
    total = np.sum(pd)
    pd = pd/total
    return pd.reshape(-1,1)

In [7]:
prob_dist(k=3)

array([[0.59733992],
       [0.38108465],
       [0.02157543]])

*initialise_strategy(lb = -1, ub = 1, n = 2, k = 2)*:
- Input: $k$ number of points with lower bound (lb), upper bound (ub) of each coordinate in $\mathbb{R}^n$ space
- Output: p_vector (distribution of k points) and v_vector (k points in $\mathbb{R}^n$)

In [8]:
def initialise_strategy(lb = -1, ub = 1, n = 2, k = 3):
    v_vector = random_points_list(lb, ub, n, k)
    p_vector = prob_dist(k)
    return [p_vector, v_vector]

In [9]:
initialise_strategy(lb = -1, ub = 1, n = 2, k = 3)

[array([[0.13891189],
        [0.65755772],
        [0.20353038]]), array([[-0.5430657 , -0.7092135 ],
        [ 0.83741406, -0.31068323],
        [ 0.28318658,  0.48041206]])]

Take for example: Generate 2 players' strategies: (p,v) and (q,w) with k = 2 servers each. c = 9 Customers in $[-1,1]^2$ space

In [10]:
[p,v] = initialise_strategy(lb = -1, ub = 1, n = 2, k = 2)
C = random_points_list(lb=-1, ub=1, n=2, k = 9)
print("Player's strategies distribution: " + np.array2string(p) + " with servers at " + np.array2string(v) + ".")
print("The customers are located at " + np.array2string(C) + ".")

Player's strategies distribution: [[0.20601975]
 [0.79398025]] with servers at [[ 0.82173541  0.19683135]
 [-0.86356042  0.49974125]].
The customers are located at [[ 0.61991361  0.41564386]
 [ 0.41347701  0.69279295]
 [ 0.14948296  0.28020492]
 [-0.94489617 -0.05479803]
 [ 0.05892316  0.18230907]
 [-0.73431443  0.65519861]
 [-0.87422808  0.91122853]
 [-0.10962088 -0.65909204]
 [ 0.87041406 -0.43653915]].


------------------

### 2.2 Necessary functions for PSRO algorithm

*Nash_eq(A)*:
- Input: matrix A of a population
- Output: Nash equilibrium p

In [11]:
def Nash_eq(A):
    '''Input: matrix A of a population
       Output: Nash equilibrium
       Note that A need to be antisymmetric for this function to generate the right Nash p'''
    n = A.shape[0]
    A_ub = -A.T
    b_ub = np.zeros((n,))
    A_eq = np.ones((1,n))
    b_eq = 1
    soln = linprog(c = np.zeros((n,)), A_ub = A_ub, b_ub = b_ub, A_eq = A_eq, b_eq = b_eq, bounds = (0,1),method='interior-point')
    return soln.x

RPS example:

In [12]:
A = np.array([[0,1,-1],[-1,0,1],[1,-1,0]])
print(Nash_eq(A))

[0.33333333 0.33333333 0.33333333]


C:\Users\jinxi\Anaconda3\lib\site-packages\scipy\optimize\_linprog_ip.py:1262: OptimizeWarning: Solving system with option 'sym_pos':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'sym_pos' to False.
  OptimizeWarning)
C:\Users\jinxi\Anaconda3\lib\site-packages\scipy\optimize\_linprog_ip.py:1274: OptimizeWarning: Solving system with option 'sym_pos':False failed. This may happen occasionally, especially as the solution is approached. However, if you see this frequently, your problem may be numerically challenging. If you cannot improve the formulation, consider setting 'lstsq' to True. Consider also setting `presolve` to True, if it is not already.
  OptimizeWarning)


*eval_matrix_1pop(popn)*:
- Input: list of strategies from 1 population of strategies where each strategy = $[p^i,v^i]$
- Output: evaluation matrix A (as described in Section 2 of the paper)

In [13]:
def eval_matrix_1pop(popn, C, k, c):
    '''Input: 1 population of strategies
       Output: evaluation matrix of that population'''
    n = len(popn)
    matrix = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            p = popn[i][0]
            v = popn[i][1]
            q = popn[j][0]
            w = popn[j][1]
            soft_v, soft_w, dist_v, dist_w = softmax_fn(v,w,C,k,c)
            matrix[i][j] = payoff(p, soft_v, q, soft_w)
    return matrix

*eval_matrix_2pop(popn)*:
- Input: lists of strategies from 2 population of strategies where each strategy = $[p^i,v^i]$ (no. of strategies from population A can differ from population B's)
- Output: evaluation matrix A (as described in Section 2 of the paper)

In [14]:
# def eval_matrix_2pop(popn1,popn2):
#     '''Input: 2 population of strategies
#        Output: evaluation matrix of those 2 populations'''
#     n = len(popn1)
#     m = len(popn2)
#     matrix = np.zeros((n,m))
#     for i in range(n):
#         for j in range(m):
#             matrix[i][j] = payoff(popn1[i],popn2[j])
#     return matrix

In [15]:
def softmax_fn(v,w,C,k,c):
    dist_v=np.zeros((c,k));
    dist_w=np.zeros((c,k));
    soft_v=np.zeros((c,k));
    soft_w=np.zeros((c,k));
    for i in range(c):
        for j in range(k):
            dist_v[i][j]=np.exp(-np.linalg.norm(C[i,:]-v[j,:])**2)
            dist_w[i][j]=np.exp(-np.linalg.norm(C[i,:]-w[j,:])**2)
        soft_v[i,:]=dist_v[i,:]/(np.sum(dist_v[i,:])+np.sum(dist_w[i,:]));
        soft_w[i,:]=dist_w[i,:]/(np.sum(dist_v[i,:])+np.sum(dist_w[i,:]));
    return soft_v,soft_w,dist_v,dist_w     

In [16]:
def transform(strategy):
    new_p = strategy[0]
    p1 = new_p[0]
    p2 = new_p[1]
    v1 = strategy[1][0]
    v2 = strategy[1][1]
    b = p1*v1 + p2*v2
    c1 = v1 - b
    c2 = v2 - b
    final_a1 = 0
    final_a2 = 0
    a2_1 = (2 -(c1+c2))/(1-p2/p1)
    a1_1 = (-p2/p1)*a2_1
    a2_2 = -(2 + (c1-c2))/(1+p2/p1)
    a1_2 = (-p2/p1)*a2_2
    a2_3 = (2+(c2-c1))/(1+p2/p1)
    a1_3 = (-p2/p1)*a2_3
    a2_4 = (c1+c2-2)/(1-p2/p1)
    a1_4 = (-p2/p1)*a2_4
    if a1_1 > c1 and a2_1 > c2:
        final_a1 = a1_1
        final_a2 = a2_1
    elif a1_2 > c1 and a2_2 < c2:
        final_a1 = a1_2
        final_a2 = a2_2
    elif a1_3 < c1 and a2_3 > c2:
        final_a1 = a1_3
        final_a2 = a2_3
    else:
        final_a1 = a1_4
        final_a2 = a2_4
    new_v1 = v1 - final_a1
    new_v2 = v2 - final_a2
    new_v = np.array([new_v1,new_v2])
    new_strategy = [new_p, new_v]
    return new_strategy

In [17]:
def payoff(p,soft_v,q,soft_w):
    payoff=np.sum(np.dot(soft_v,p)-np.dot(soft_w,q))
    return payoff

In [18]:
def oracle(popn,nash_p,eta,C,k,c):
    curr_agent_p=popn[-1][0].reshape(k,1);
    curr_agent_v=popn[-1][1];
    next_agent_p=curr_agent_p+eta*gradient_p(popn,nash_p,curr_agent_v,C,k,c);
    next_agent_p = next_agent_p/np.sum(next_agent_p)
    next_agent_v=curr_agent_v+eta*gradient_v(popn,nash_p,curr_agent_p,curr_agent_v,C,k,c);
    #print(gradient_v(popn,nash_p,curr_agent_p,curr_agent_v,C,k,c))
    return next_agent_p,next_agent_v

In [19]:
def gradient_p(popn,nash_p,agent_v,C,k,c):
    sum_vec=np.zeros((k,1));
    for i in range(len(popn)):
        temp_agent_p=popn[i][0].reshape(k,1);
        temp_agent_v=popn[i][1];
        temp_v,temp_w,tempdist_v,tempdist_w=softmax_fn(agent_v,temp_agent_v,C,k,c);
        val_grad=np.sum(temp_v,axis=0).reshape(k,1) #sum the rows
        sum_vec=sum_vec+nash_p[i]*val_grad; #multiply w.r.t Nash p.
        
    return sum_vec; 

In [28]:
def gradient_v(popn,nash_p,agent_p,agent_v,C,k,c): #agent_p, agent_v are strategies of the current agent "v_t of popn B_t"
    sum_mat=np.zeros((k,3));
    for i in range(len(popn)):
        temp_agent_p=popn[i][0].reshape(k,1);
        temp_agent_v=popn[i][1];
        temp_v,temp_w,tempdist_v,tempdist_w=softmax_fn(agent_v,temp_agent_v,C,k,c);
        val_grad_mat=calc_gradient_matrix(agent_p,agent_v,temp_agent_p,temp_agent_v,temp_v,temp_w,tempdist_v,tempdist_w,C,k,c) #sum the rows
        #To add the remaining parameters to calc_gradient_matrix function-See below.
        sum_mat=sum_mat+nash_p[i]*val_grad_mat; #multiply w.r.t Nash p.
        
    return sum_mat

In [29]:
def calc_gradient_matrix(agent_p,agent_v,temp_agent_p,temp_agent_v,temp_v,temp_w,tempdist_v,tempdist_w,C,k,c):
    #agent_p - p vector of agent v_t
    #temp_agent_q- q vector of the "current opposition agent"
    #W- location matrix of "current opposition agent"
    #V- location matrix of agent v_t
    grad_mat=np.zeros((k,3));
    V=agent_v;
    W=temp_agent_v;
    for j in range(k):
        S_v=0;
        S_w=0;
        for i in range(c):
            for l in range(k):
                v_loc=V[j,:];
                S_v=S_v+agent_p[j]*grad_payoff(v_loc,j,l,1,i,temp_v,temp_w,tempdist_v,tempdist_w,C,k,c);
                S_w=S_w+temp_agent_p[j]*grad_payoff(v_loc,j,l,0,i,temp_v,temp_w,tempdist_v,tempdist_w,C,k,c);
        
         
        grad_mat[j,:]=S_v-S_w;  
      
    return grad_mat    

In [22]:
def grad_payoff(v_loc,v_loc_idx,w_loc_idx,agent_bool,c_idx,temp_v,temp_w,dist_v,dist_w,C,k,c): #Gradient with respect to v_loc a location strategy of agent v.
    
    if(agent_bool==1): #Differentiating agent_v
        
        if(v_loc_idx==w_loc_idx):
            grad=(2*(C[c_idx,:]-v_loc)*dist_v[c_idx,v_loc_idx]*(np.sum(dist_v[c_idx,:])+np.sum(dist_w[c_idx,:])-dist_v[c_idx,v_loc_idx]))/(np.sum(dist_v[c_idx,:])+np.sum(dist_w[c_idx,:]))**2
        else:
            grad=(-2*(C[c_idx,:]-v_loc)*dist_v[c_idx,v_loc_idx]*dist_v[c_idx,w_loc_idx])/(np.sum(dist_v[c_idx,:])+np.sum(dist_w[c_idx,:]))**2
    
    else:
#         if(v_loc_idx==w_loc_idx): #Differentiating agent_w
#             grad=(2*(C[c_idx,:]-v_loc)*dist_v[c_idx,v_loc_idx]*(np.sum(dist_v[c_idx,:])+np.sum(dist_w[c_idx,:])-dist_w[c_idx,v_loc_idx]))/(np.sum(dist_v[c_idx,:])+np.sum(dist_w[c_idx,:]))**2
#         else:
        grad=(-2*(C[c_idx,:]-v_loc)*dist_v[c_idx,v_loc_idx]*dist_w[c_idx,w_loc_idx])/(np.sum(dist_v[c_idx,:])+np.sum(dist_w[c_idx,:]))**2   
    
    return grad    

-----------------

### 2.3 PSRO$_N$ and Self-play Implementation

In [23]:
def PSRO_N(popn, C, k, c, eta = 1, epoch = 5):
    strategies = popn
    n = len(strategies)
    for i in range(epoch):
        A = eval_matrix_1pop(strategies, C, k, c)
        nash_p = Nash_eq(A)
        print(nash_p)
        new_p, new_v = oracle(strategies,nash_p,eta,C,k,c)
        new_strategy = [new_p, new_v]
        strategies.append(new_strategy)
        n = n + 1
        print("There are " + str(n) + " strategies in the population now.")
    return strategies

In [24]:
def Self_play(popn, C, k, c, eta = 1, epoch = 5):
    strategies =  popn
    n = len(strategies)
    for i in range(epoch):
        nash_p = np.zeros((n,1))
        nash_p[-1] = 1
        print(nash_p)
        new_p, new_v = oracle(strategies,nash_p,eta,C,k,c)
        new_strategy = [new_p, new_v]
        strategies.append(new_strategy)
        n = n + 1
        print("There are " + str(n) + " strategies in the population now.")
    return strategies

--------------

### 2.4 Population Performance Measures

*rel_popn_perf(popn1, popn2)*:
- Input: lists of strategies from 2 population of strategies where each strategy = $[p^i,v^i]$ (no. of strategies from population A can differ from population B's)
- Output: relative population performance (defined in Definition 3 of Section 3.1 in th paper)

In [25]:
def rel_popn_perf(popn1, popn2):
    perf = 0
    A_1 = eval_matrix_1pop(popn1)
    A_2 = eval_matrix_1pop(popn2)
    A_12 = eval_matrix_2pop(popn1,popn2)
    p = Nash_eq(A_1).reshape(-1,1)
    q = Nash_eq(A_2).reshape(-1,1)
    perf = np.dot(np.dot(p.T,A_12),q)
    return perf

*eff_diversity(popn)*:
- Input: list of strategies from 1 population of strategies where each strategy = $[p^i,v^i]$
- Output: effective diversity of the population (defined in Definition 4 of Section 3.2 in the paper)

In [26]:
def eff_diversity(popn):
    d = 0
    A = eval_matrix_1pop(popn)
    p = Nash_eq(A).reshape(-1,1)
    A_plus = np.clip(A,0,None)
    d = np.dot(np.dot(p.T,A_plus),p)
    return d

--------------

## 3. Running all implementations

### 3.0 Initialisation of population and customers

**NOTE: DO NOT RUN THE CODE BELOW!!!**
--------

----------

In [ ]:
lb = -1 #lower bound of the space
ub = 1 #upper bound of the space
n = 2 #dimension of the space
c = 9 #number of customers

strategy1_3s = initialise_strategy(lb, ub, n, k=3) # 1st strategy for 3 servers
strategy1_9s = initialise_strategy(lb, ub, n, k=9) # 1st strategy for 9 servers
strategy1_15s = initialise_strategy(lb, ub, n, k=15) # 1st strategy for 15 servers

popn_3s = [strategy1_3s]
popn_9s = [strategy1_9s]
popn_15s = [strategy1_15s]

C = random_points_list(lb, ub, n, k = c)

initialisation = open('initialisation.pkl','wb')
pickle.dump(popn_3s, initialisation)
pickle.dump(popn_9s, initialisation)
pickle.dump(popn_15s, initialisation)
pickle.dump(C, initialisation)
initialisation.close()

In [27]:
lb = -1 #lower bound of the space
ub = 1 #upper bound of the space
n = 3 #dimension of the space
c = 9 #number of customers

C = random_points_list(lb, ub, n, k = c)

strategy1_3s = initialise_strategy(lb, ub, n, k=3)
strategy2_3s = initialise_strategy(lb, ub, n, k=3)
strategy3_3s = initialise_strategy(lb, ub, n, k=3)

popn = [strategy1_3s, strategy2_3s, strategy3_3s]

print(popn)

A = eval_matrix_1pop(popn, C, k=3, c=9)
nash = Nash_eq(A)
print(A)
print(nash)

[[array([[0.6557327 ],
       [0.24911586],
       [0.09515144]]), array([[-0.61266104,  0.34266345, -0.44932756],
       [ 0.03288825,  0.02997487,  0.49965622],
       [-0.2304317 ,  0.82405776,  0.38548567]])], [array([[0.08460683],
       [0.0470628 ],
       [0.86833037]]), array([[ 0.72626966, -0.92232135, -0.00807714],
       [ 0.6639518 , -0.34556372, -0.10352617],
       [ 0.94207596, -0.87770516, -0.05187619]])], [array([[0.51207664],
       [0.3823197 ],
       [0.10560366]]), array([[ 0.97060517, -0.94577817, -0.5704307 ],
       [ 0.83210709,  0.43950315,  0.02108286],
       [ 0.3978626 , -0.61694941, -0.59963332]])]]
[[ 0.          0.34019729  0.208113  ]
 [-0.34019729  0.         -0.20960614]
 [-0.208113    0.20960614  0.        ]]
[1.00000000e+00 1.01843046e-10 4.17931868e-12]


In [30]:
new_popn = PSRO_N(popn = popn, C = C, k=3, c = 9, eta = 1,epoch = 20)

[1.00000000e+00 1.01843046e-10 4.17931868e-12]
There are 4 strategies in the population now.
[3.20180208e-11 3.74477726e-11 2.88782089e-11 1.00000000e+00]
There are 5 strategies in the population now.
[1.91552819e-12 1.58054588e-13 7.43354496e-13 9.13224686e-14
 1.00000000e+00]
There are 6 strategies in the population now.
[1.38960859e-09 4.22057544e-10 7.29225119e-10 5.11458831e-09
 8.86254029e-10 9.99999996e-01]
There are 7 strategies in the population now.
[4.51897598e-09 1.73758536e-09 2.72590225e-09 1.03792430e-08
 2.17950960e-07 5.36944342e-09 9.99999786e-01]
There are 8 strategies in the population now.
[2.89581218e-11 1.98791224e-11 2.41191622e-11 4.17451628e-11
 9.49246535e-10 5.65112449e-11 7.21146369e-09 9.99999992e-01]
There are 9 strategies in the population now.
[1.53800672e-09 5.79186813e-10 9.98708008e-10 4.05739816e-09
 2.95031274e-08 1.06513754e-07 2.70982410e-07 1.06094361e-08
 9.99999591e-01]
There are 10 strategies in the population now.
[2.63248171e-11 1.13544485e

In [55]:
np.sum(soft_v[0]+soft_w[0])

0.9999999999999999

---------

**JUST RUN THIS TO LOAD THE INITIAL POPULATIONS AND FIXED CUSTOMERS**
----------

In [13]:
lb = -1 #lower bound of the space
ub = 1 #upper bound of the space
n = 2 #dimension of the space
c = 9 #number of customers

initialisation = open('initialisation.pkl', 'rb')
popn_3s = pickle.load(initialisation)
popn_9s = pickle.load(initialisation)
popn_15s = pickle.load(initialisation)
C = pickle.load(initialisation)
initialisation.close()
print("1st strategy for 3 servers:")
print(popn_3s)
print("1st strategy for 9 servers:")
print(popn_9s)
print("1st strategy for 15 servers:")
print(popn_15s)
print("The fixed 9 customers are:")
print(C)

1st strategy for 3 servers:
[[array([[0.33690418],
       [0.57266755],
       [0.09042826]]), array([[-0.6588831 , -0.30443716],
       [ 0.63009139,  0.84892945],
       [-0.20919258, -0.83122018]])]]
1st strategy for 9 servers:
[[array([[0.12651314],
       [0.20330865],
       [0.28024899],
       [0.02860374],
       [0.09365179],
       [0.03538327],
       [0.05292266],
       [0.08309388],
       [0.09627388]]), array([[ 0.559884  ,  0.18746408],
       [-0.43187223,  0.72890129],
       [ 0.8663807 ,  0.49655679],
       [ 0.49771878,  0.69660568],
       [-0.35238418, -0.60426657],
       [-0.52917142, -0.54600663],
       [ 0.81011878,  0.39529413],
       [-0.26856388, -0.06780096],
       [-0.90123934, -0.82303664]])]]
1st strategy for 15 servers:
[[array([[0.08734612],
       [0.04345478],
       [0.06277414],
       [0.10801926],
       [0.04284768],
       [0.00301178],
       [0.06298912],
       [0.06446927],
       [0.09097309],
       [0.085604  ],
       [0.0713737

----------

### 3.1 Self-play

**NOTE: DO NOT RUN THE CODE BELOW!!!**
----------

-------------

In [ ]:
lb = -1 #lower bound of the space
ub = 1 #upper bound of the space
n = 2 #dimension of the space
c = 9 #number of customers

initialisation = open('initialisation.pkl', 'rb')
popn_3s = pickle.load(initialisation)
popn_9s = pickle.load(initialisation)
popn_15s = pickle.load(initialisation)
C = pickle.load(initialisation)
initialisation.close()
print("1st strategy for 3 servers:")
print(popn_3s)
print("1st strategy for 9 servers:")
print(popn_9s)
print("1st strategy for 15 servers:")
print(popn_15s)
print("The fixed 9 customers are:")
print(C)

In [ ]:
popn_self_play = open('popn_self_play.pkl','wb')

In [ ]:
popn_self_play_3s = Self_play(popn = popn_3s, C = C, k=3, c = 9, eta = 1,epoch = 100)
pickle.dump(popn_self_play_3s, popn_self_play)

In [ ]:
popn_self_play_9s = Self_play(popn = popn_9s, C = C, k=9, c = 9, eta = 1,epoch = 100)
pickle.dump(popn_self_play_9s, popn_self_play)

In [ ]:
popn_self_play_15s = Self_play(popn = popn_15s, C = C, k=15, c = 9, eta = 1,epoch = 100)
pickle.dump(popn_self_play_15s, popn_self_play)

In [ ]:
popn_self_play.close()

-------------

**JUST RUN THIS TO LOAD THE STRATEGIES OF ALL SELF-PLAY POPULATIONS**
----------

In [ ]:
popn_self_play = open('popn_self_play.pkl', 'rb')
popn_self_play_3s = pickle.load(popn_self_play)
popn_self_play_9s = pickle.load(popn_self_play)
popn_self_play_15s = pickle.load(popn_self_play)
popn_self_play.close()
print(len(popn_self_play_3s), len(popn_self_play_9s), len(popn_self_play_15s))
print("Last strategy for 3 servers:")
print(popn_self_play_3s[-1])
print("Last strategy for 9 servers:")
print(popn_self_play_9s[-1])
print("Last strategy for 15 servers:")
print(popn_self_play_15s[-1])

The following plots how the phi value change as a new strategy is trained for 100 epoch.

In [ ]:
phi_changes_3s = []
for i in range(len(popn_self_play_3s)-1):
    soft_v, soft_w, _, _ = softmax_fn(popn_self_play_3s[i+1][1],popn_self_play_3s[i][1],C,k=3,c=9)
    phi = payoff(popn_self_play_3s[i+1][0],soft_v,popn_self_play_3s[i][0],soft_w) 
    phi_changes_3s.append(phi)

In [ ]:
plt.style.use('seaborn')
plt.plot(range(len(phi_changes_3s)), phi_changes_3s)
plt.xlabel('Epoch')
plt.ylabel('Phi Value')
plt.show()

--------------------

### 3.2 PSRO$_N$

**NOTE: DO NOT RUN THE CODE BELOW!!!**
----------

-----------

In [ ]:
lb = -1 #lower bound of the space
ub = 1 #upper bound of the space
n = 2 #dimension of the space
c = 9 #number of customers

initialisation = open('initialisation.pkl', 'rb')
popn_3s = pickle.load(initialisation)
popn_9s = pickle.load(initialisation)
popn_15s = pickle.load(initialisation)
C = pickle.load(initialisation)
initialisation.close()
print("1st strategy for 3 servers:")
print(popn_3s)
print("1st strategy for 9 servers:")
print(popn_9s)
print("1st strategy for 15 servers:")
print(popn_15s)
print("The fixed 9 customers are:")
print(C)

In [ ]:
popn_psro_n = open('popn_psro_n.pkl','wb')

In [ ]:
popn_psro_n_3s = PSRO_N(popn = popn_3s, C = C, k=3, c = 9, eta = 1,epoch = 50)
pickle.dump(popn_psro_n_3s, popn_psro_n)

In [ ]:
popn_psro_n_9s = PSRO_N(popn = popn_9s, C = C, k=9, c = 9, eta = 1,epoch = 100)
pickle.dump(popn_psro_n_9s, popn_psro_n)

In [ ]:
popn_psro_n_15s = PSRO_N(popn = popn_15s, C = C, k=15, c = 9, eta = 1,epoch = 100)
pickle.dump(popn_psro_n_15s, popn_psro_n)

In [ ]:
popn_psro_n.close()

------------

**JUST RUN THIS TO LOAD THE STRATEGIES OF ALL PSRO$_N$ POPULATIONS**
----------

In [2]:
popn_psro_n = open('popn_psro_n.pkl', 'rb')
popn_psro_n_3s = pickle.load(popn_psro_n)
popn_psro_n_9s = pickle.load(popn_psro_n)
popn_psro_n_15s = pickle.load(popn_psro_n)
popn_psro_n.close()
print(len(popn_psro_n_3s), len(popn_psro_n_9s), len(popn_psro_n_15s))
# print("Last strategy for 3 servers:")
# print(popn_psro_n_3s[-1])
# print("Last strategy for 9 servers:")
# print(popn_psro_n_9s[-1])
# print("Last strategy for 15 servers:")
# print(popn_psro_n_15s[-1])

101 101 101


In [ ]:
phi_changes_3s_psro_n = []
for i in range(len(popn_psro_n_3s)-1):
    soft_v, soft_w, _, _ = softmax_fn(popn_psro_n_3s[i+1][1],popn_psro_n_3s[i][1],C,k=3,c=9)
    phi = payoff(popn_psro_n_3s[i+1][0],soft_v,popn_psro_n_3s[i][0],soft_w) 
    phi_changes_3s_psro_n.append(phi)

In [ ]:
plt.style.use('seaborn')
plt.plot(range(len(phi_changes_3s_psro_n)), phi_changes_3s_psro_n)
plt.xlabel('Epoch')
plt.ylabel('Phi Value Changes')
plt.show()

-------

In [ ]:
A_psro_n_3s = eval_matrix_1pop(popn_psro_n_3s, C, k=3, c=9)
nash_psro_n_3s = Nash_eq(A_psro_n_3s)
extra = 1-sum(nash_psro_n_3s)
nash_psro_n_3s[-1] += extra
print(sum(nash_psro_n_3s))

In [ ]:
sample_psro_n_3s = []
sample_index = np.random.choice(101, 1000, p=nash_psro_n_3s)
for i in range(1000):
    index = sample_index[i]
    sample_psro_n_3s.append(popn_psro_n_3s[index])
print(len(sample_psro_n_3s))

In [ ]:
self_play_3s_best = popn_self_play_3s[-1]
phi_compare_3s = []
for i in range(1000):
    psro_n_sample = sample_psro_n_3s[i]
    soft_v, soft_w, _, _ = softmax_fn(psro_n_sample[1],self_play_3s_best[1],C,k=3,c=9)
    phi = payoff(psro_n_sample[0],soft_v,self_play_3s_best[0],soft_w) 
    phi_compare_3s.append(phi)

In [ ]:
plt.style.use('seaborn')
plt.plot(range(len(phi_compare_3s)), phi_compare_3s)
plt.xlabel('Epoch')
plt.ylabel('Phi Value Comparison')
plt.show()

------------

# Extra

In [ ]:
lb = -0.25 #lower bound of the space
ub = 0.25 #upper bound of the space
n = 2 #dimension of the space
k = 4 #number of servers
c = 4 #number of customers

strategy1 = initialise_strategy(lb, ub, n, k=4)
#strategy2 = initialise_strategy(lb, ub, n, k=4)
strategy1[0] = np.array([0.25,0.25,0.25,0.25])
strategy1[1] = np.array([[0.7,0.7],[-0.7,0.7],[0.7,-0.7],[-0.7,-0.7]])
#strategy2[0] = np.array([0.25,0.25,0.25,0.25])
popn = [strategy1]#,strategy2]

C = np.array([[-0.8,0.8],[-0.8,-0.8],[0.8,-0.8],[0.8,0.8]])
print(popn)
print(C)
#initial_p = prob_dist(k=3)

In [ ]:
popn_psro_n = PSRO_N(popn, C, k, c,eta = 1, epoch = 50)

In [ ]:
popn_psro_n

In [ ]:
A = eval_matrix_1pop(popn_psro_n, C, k, c)
nash = Nash_eq(A)
print(A)
print(nash)

----------

In [ ]:
lb = -1 #lower bound of the space
ub = 1 #upper bound of the space
n = 2 #dimension of the space
k = 3 #number of servers
c = 9 #number of customers

#strategy1_2s = initialise_strategy(lb, ub, n, k=2)
strategy1_3s = initialise_strategy(lb, ub, n, k=3)
#strategy1_4s = initialise_strategy(lb, ub, n, k=4)
#strategy1_5s = initialise_strategy(lb, ub, n, k=5)
# strategy2 = initialise_strategy(lb, ub, n, k)
# strategy3 = initialise_strategy(lb, ub, n, k)
# strategy4 = initialise_strategy(lb, ub, n, k)
# strategy5 = initialise_strategy(lb, ub, n, k)
# strategy6 = initialise_strategy(lb, ub, n, k)
# strategy7 = initialise_strategy(lb, ub, n, k)
# strategy8 = initialise_strategy(lb, ub, n, k)
#popn_2s = [strategy1_2s]
popn_3s = [strategy1_3s]
#popn_4s = [strategy1_4s]
#popn_5s = [strategy1_5s]
#, strategy2, strategy3, strategy4] #, strategy5, strategy6, strategy7, strategy8]
customers = open('customers.pkl', 'rb')
C = pickle.load(customers)
customers.close()
print(popn_3s)
print(C)
#initial_p = prob_dist(k=3)

In [ ]:
popn_psro_n_3s = PSRO_N(popn_3s, C, k, c,eta = 1, epoch = 50)

In [ ]:
popn_psro_n_3s[-1]

In [ ]:
A = eval_matrix_1pop(popn_psro_n_3s, C, k, c)
nash = Nash_eq(A)
print(A)
print(nash)

In [ ]:
np.sum(nash)

In [ ]:
popn_self_play_3s = Self_play(popn = popn_3s, C = C, k=3, c = c, eta = 0.05,epoch = 10)

In [ ]:
popn_self_play_3s[-1]